In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
import os

In [2]:
# --- CONFIGURATION ---
CSV_PATH = "dataset/e_waste_classification_dataset.csv"  # <--- CHANGE THIS to your actual CSV path
MODEL_SAVE_PATH = "trained_models/text_classifier.pkl"


In [3]:
import os
import pandas as pd

# Create sample dataset if file doesn't exist
if not os.path.exists(CSV_PATH):
    os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
    sample_data = {
        'waste_description': ['broken phone', 'old laptop', 'damaged tablet', 'faulty charger'],
        'waste_category': ['electronics', 'electronics', 'electronics', 'electronics']
    }
    pd.DataFrame(sample_data).to_csv(CSV_PATH, index=False)
    print(f"⚠️ Created sample dataset at {CSV_PATH}")

print(f"Loading data from {CSV_PATH}...")
df = pd.read_csv(CSV_PATH)

print("Found columns:", df.columns.tolist())

# Rename columns to expected names
df = df.rename(columns={
    "waste_description": "text",
    "waste_category": "category"
})

# Validate again
required_cols = {"text", "category"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"CSV must contain columns: {required_cols}")

# Handle missing values
df = df.dropna(subset=["text", "category"])

print(f"✅ Loaded {len(df)} rows of data.")


Loading data from dataset/e_waste_classification_dataset.csv...
Found columns: ['id', 'waste_description', 'waste_category']
✅ Loaded 1000 rows of data.


In [4]:
# --- 2. SPLIT DATA (The Missing Part) ---
# We use 80% of data to Train, and 20% to Test
from sklearn.model_selection import train_test_split
print("Label distribution BEFORE split:")
print(df['category'].value_counts())
print("Unique labels:", df['category'].unique())
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], 
    df['category'], 
    test_size=0.2, 
    random_state=42
)

print(f"📊 Training on {len(X_train)} samples, Testing on {len(X_test)} samples.")

Label distribution BEFORE split:
category
Hazardous & Lamps               258
Large Equipment                 256
Screens & Monitors              252
Small IT & General Equipment    234
Name: count, dtype: int64
Unique labels: ['Hazardous & Lamps' 'Large Equipment' 'Screens & Monitors'
 'Small IT & General Equipment']
📊 Training on 800 samples, Testing on 200 samples.


In [5]:
# --- 2. BUILD PIPELINE ---
from sklearn.metrics import accuracy_score, classification_report

text_pipeline = Pipeline([
    (
        'tfidf',
        TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 2),   # better context
            min_df=2              # ignore ultra-rare words
        )
    ),
    (
        'clf',
        LogisticRegression(
            max_iter=300,
            class_weight='balanced',  # 🔥 handles class imbalance
            solver='lbfgs'
        )
    )
])

# --- 3. TRAIN ---
print("Training Text Model...")


text_pipeline.fit(X_train, y_train)

# --- 4. TEST ---
print("Testing Model...")
predictions = text_pipeline.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"\n✅ Model Accuracy: {accuracy * 100:.2f}%")

print("\nDetailed Classification Report:")
print(classification_report(y_test, predictions))

# --- 5. FINAL TRAIN ON FULL DATASET ---
print("Saving final model (trained on full dataset)...")
text_pipeline.fit(df['text'], df['category'])

# --- 6. SAVE ---
import os, joblib

MODEL_SAVE_PATH = "trained_models/text_waste_classifier.joblib"
os.makedirs("trained_models", exist_ok=True)

joblib.dump(text_pipeline, MODEL_SAVE_PATH)
print(f"✅ Text Model Saved as {MODEL_SAVE_PATH}")

Training Text Model...
Testing Model...

✅ Model Accuracy: 100.00%

Detailed Classification Report:
                              precision    recall  f1-score   support

           Hazardous & Lamps       1.00      1.00      1.00        45
             Large Equipment       1.00      1.00      1.00        62
          Screens & Monitors       1.00      1.00      1.00        36
Small IT & General Equipment       1.00      1.00      1.00        57

                    accuracy                           1.00       200
                   macro avg       1.00      1.00      1.00       200
                weighted avg       1.00      1.00      1.00       200

Saving final model (trained on full dataset)...
✅ Text Model Saved as trained_models/text_waste_classifier.joblib
